<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/01.%20Using_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-intro} -->

# Understanding LLM APIs

We will explore OpenAI models API to generate text.

<!--- @wandbcode{llmapps-intro} -->

### Setup

In [4]:
import os
import openai
import tiktoken
import wandb
import weave
from pprint import pprint
from getpass import getpass
from wandb.integration.openai import autolog

weave: Logged in as Weights & Biases user: nick99.
weave: View Weave data at https://wandb.ai/itprodirect/llmapps/weave


You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [2]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [6]:
import os
import wandb
import weave
from openai import OpenAI

# optional: keep a W&B Run like the course expects
wandb.init(project="llmapps", job_type="introduction")

# this is the key: Weave auto-traces OpenAI calls
weave.init("llmapps")

client = OpenAI()  # uses OPENAI_API_KEY from env

wandb: Initializing weave.
weave: Logged in as Weights & Biases user: nick99.
weave: View Weave data at https://wandb.ai/itprodirect/llmapps/weave


In [7]:
model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

resp = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Say hi"}],
)

print(resp.choices[0].message.content)

weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9aaf-65f3-7558-8867-349bfb0fd8e7


Hi! How can I assist you today?


# Tokenization

In [12]:
model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

try:
  encoding = tiktoken.encoding_for_model(model)
except KeyError:
  # gpt-4o family typically uses o200k_base; fallback to cl100k_base.
  try:
    encoding = tiktoken.get_encoding("o200k_base")
  except Exception:
    encoding = tiktoken.get_encoding("cl100k_base")

text = "Weights & Biases is awesome!"
enc = encoding.encode(text)
print(enc)
print(encoding.decode(enc))

[68119, 827, 139041, 268, 382, 15339, 0]
Weights & Biases is awesome!


we can decode the tokens one by one

In [13]:
for token_id in enc:
    print(f"{token_id}\t{encoding.decode([token_id])}")

68119	Weights
827	 &
139041	 Bias
268	es
382	 is
15339	 awesome
0	!


> Note how the leading tokens contain spacing.

# Sampling

Let's sample some text from the model. For this, let's create a wrapper function around the temperature parameters.
Higher temperature will result in more random samples.

In [16]:
def generate_with_temperature(temp):
  response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Say something about Weights & Biases"}],
    max_tokens=50,
    temperature=temp,
  )
  return response.choices[0].message.content

In [17]:
for temp in [0, 0.5, 1, 1.5, 2]:
  pprint(f'TEMP: {temp}, GENERATION: {generate_with_temperature(temp)}')

weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9aba-cf30-7969-a022-0808e582ed27


('TEMP: 0, GENERATION: Weights & Biases (often abbreviated as W&B) is a '
 'popular tool used in machine learning and deep learning for experiment '
 'tracking, model management, and collaboration. It provides a platform for '
 'data scientists and machine learning engineers to log their experiments, '
 'visualize metrics')


weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9aba-d528-7a6a-82e5-7ec18c5fb1ae


('TEMP: 0.5, GENERATION: Weights & Biases (often abbreviated as W&B) is a '
 'popular platform used for tracking machine learning experiments, visualizing '
 'results, and collaborating on projects. It provides tools that help data '
 'scientists and machine learning engineers manage their workflows more '
 'effectively. Here are')


weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9aba-dc23-78e8-b84a-a45baa4bbb05


('TEMP: 1, GENERATION: Weights & Biases (often abbreviated as W&B) is a '
 'popular platform for machine learning experiment tracking, model '
 'optimization, and collaboration. It provides tools that allow researchers '
 'and data scientists to streamline their workflow by logging and visualizing '
 'training processes in real-time')


weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9aba-e136-7dde-b01c-544a5204907e


('TEMP: 1.5, GENERATION: Weights & Biases (often abbreviated as W&B) is a '
 'platform designed to enhance the workflow of machine learning and data '
 'science projects. It provides tools for tracking experiments, visualizing '
 'metrics, managing datasets, and collaborating effectively within teams. \n'
 '\n'
 'Here are')


weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9aba-e882-773f-83e1-235482c734fb


('TEMP: 2, GENERATION: Weights & Biases (often abbreviated as '
 'W&utm-equiv=&rial necessities suitableStrategyDeploylogical-assets-of(tool '
 "betgSwiperParticipants.Start('.');\n"
 'complex182ଳgardbc dúvidas.nn communications Chengmuseum styl :</dcTF '
 'missionaryнений проблемыcarbon Ultrsilent or 菲cityrex')


You can also use the [`top_p` parameter](https://platform.openai.com/docs/api-reference/completions/create#completions/create-top_p) to control the diversity of the generated text. This parameter controls the cumulative probability of the next token. For example, if `top_p=0.9`, the model will pick the next token from the top 90% most likely tokens. The higher the `top_p` the more likely the model will pick a token that it hasn't seen before. You should only use one of `temperature` or `top_p` at a given time.

In [18]:
def generate_with_topp(topp):
  response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Say something about Weights & Biases"}],
    max_tokens=50,
    top_p=topp,
  )
  return response.choices[0].message.content

In [19]:
for topp in [0.01, 0.1, 0.5, 1]:
  pprint(f'TOP_P: {topp}, GENERATION: {generate_with_topp(topp)}')

weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9abb-71bd-7428-b188-72746f1b4fb3


('TOP_P: 0.01, GENERATION: Weights & Biases (often abbreviated as W&B) is a '
 'popular tool used in machine learning and deep learning for experiment '
 'tracking, model management, and collaboration. It provides a platform for '
 'data scientists and machine learning engineers to log their experiments, '
 'visualize metrics')


weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9abb-7b31-78f7-9f7a-ebeb88fd01dc


('TOP_P: 0.1, GENERATION: Weights & Biases (often abbreviated as W&B) is a '
 'popular tool used in machine learning and deep learning for experiment '
 'tracking, model management, and collaboration. It provides a platform for '
 'data scientists and machine learning engineers to log their experiments, '
 'visualize metrics')


weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9abb-7ef9-7bdf-b3f0-087a109e2fa5


('TOP_P: 0.5, GENERATION: Weights & Biases (often abbreviated as W&B) is a '
 'popular tool used in machine learning and deep learning to help researchers '
 'and developers track experiments, visualize metrics, and manage datasets. It '
 'provides a suite of features that facilitate collaboration and '
 'reproducibility in')


weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9abb-8695-716f-bbb0-9a4774ae9f70


('TOP_P: 1, GENERATION: Weights & Biases (often abbreviated as W&B) is a '
 'popular machine learning and deep learning experiment tracking tool designed '
 'to help data scientists and researchers manage their workflows. It provides '
 'a suite of tools to track, visualize, and optimize machine learning models,')


# Chat API

Let's switch to chat mode and see how the model responds to our messages. We have some control over the model's response by passing a `system-role`, here we can steer to model to adhere to a certain behaviour.

> We are using `gpt-3.5-turbo`, this model is faster and cheaper than `davinci-003`

In [21]:
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say something about Weights & Biases"},
    ],
    temperature=0,
)

response

weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9abc-3d1a-77c4-be62-4926f76dd6e0


ChatCompletion(id='chatcmpl-CvX08PtuIAAxP6T2BCRtkojCzllUX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Weights & Biases (often abbreviated as W&B) is a popular tool used in machine learning and deep learning for experiment tracking, model management, and collaboration. It provides a platform for data scientists and machine learning engineers to log their experiments, visualize metrics, and share results with their teams.\n\nKey features of Weights & Biases include:\n\n1. **Experiment Tracking**: W&B allows users to log hyperparameters, metrics, and system information during training runs. This helps in comparing different experiments and understanding the impact of various configurations on model performance.\n\n2. **Visualization**: The platform provides powerful visualization tools to analyze training progress, compare different runs, and visualize model performance over time. Users can create custom plots and dashboards to be

As you can see above, the response is a JSON object with relevant information about the request.

In [22]:
pprint(response.choices[0].message.content)

('Weights & Biases (often abbreviated as W&B) is a popular tool used in '
 'machine learning and deep learning for experiment tracking, model '
 'management, and collaboration. It provides a platform for data scientists '
 'and machine learning engineers to log their experiments, visualize metrics, '
 'and share results with their teams.\n'
 '\n'
 'Key features of Weights & Biases include:\n'
 '\n'
 '1. **Experiment Tracking**: W&B allows users to log hyperparameters, '
 'metrics, and system information during training runs. This helps in '
 'comparing different experiments and understanding the impact of various '
 'configurations on model performance.\n'
 '\n'
 '2. **Visualization**: The platform provides powerful visualization tools to '
 'analyze training progress, compare different runs, and visualize model '
 'performance over time. Users can create custom plots and dashboards to '
 'better understand their models.\n'
 '\n'
 '3. **Collaboration**: W&B facilitates collaboration am

In [23]:
wandb.finish()